In [ ]:
!pip install asyncpraw
!pip install praw
!pip install pytesseract
!pip install paddleocr
!pip install paddlepaddle

import asyncpraw
import re
import asyncio
import csv

##Reddit

In [ ]:
import asyncpraw
import re
import asyncio
from types import SimpleNamespace

# Your Reddit credentials
reddit = asyncpraw.Reddit(
    client_id='hxUFRWQlIRSoZFiGw8i7rA',
    client_secret='-RadwRcpbx2yI4g11F34MZKwPCjfVg',
    user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/134.0.0.0 Safari/537.36'
)

# Keyword labeling rules
labels = {
    'siblings': [r'>!\s*siblings\s*!<', r'\[siblings\]'],
    'dating': [r'>!\s*dating\s*!<', r'\[dating\]'],
}

# Function to check if the post is an image
def is_image_post(url):
    return url.lower().endswith(('.jpg', '.jpeg', '.png', '.gif')) or url.lower().startswith('https://preview.redd.it')

# Function to collect all comments (including replies)
async def get_all_comments(comment_forest):
    all_comments = []
    stack = list(comment_forest)

    while stack:
        comment = stack.pop()
        all_comments.append(comment)
        if hasattr(comment, "replies") and comment.replies:
            replies = comment.replies
            stack.extend(replies)

    return all_comments

# Function to label posts from OP's comment
async def label_from_op_comment(submission):
    if not is_image_post(submission.url):
        return None  # Skip non-image posts

    await submission.load()
    await submission.comments.replace_more(limit=None)

    # Get all comments and filter for OP's comments
    all_comments = await get_all_comments(submission.comments)
    op_comments = [c for c in all_comments if c.author == submission.author]

    if submission.selftext.strip():
        fake_comment = SimpleNamespace(
            body=submission.selftext,
            author=submission.author
        )
        op_comments.append(fake_comment)

    # If there are no OP comments, skip this post
    if not op_comments:
        print(f"potential error, no answer found: https://reddit.com{submission.permalink}")
        return None

    # Check if any of the OP comments contain the labels
    for comment in op_comments:
        text = comment.body.lower()

        for label, patterns in labels.items():
            for pattern in patterns:
                if re.search(pattern, text, re.IGNORECASE):
                    return {
                        'post_url': f"https://reddit.com{submission.permalink}",
                        'image_url': submission.url,
                        'label': label,
                        'op_comment': comment.body.strip()
                    }

    # If no label found, skip the post without printing
    return None

def save_to_csv(data):
    with open("reddit_posts.csv", mode='w', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=['post_url', 'image_url', 'label', 'op_comment'])
        writer.writeheader()
        writer.writerows(data)

# Main function to loop through submissions
async def main():
    subreddit = await reddit.subreddit("siblingsordating")

    result_data = []  # Store results here

    async for submission in subreddit.new(limit=100000):
        labeled_data = await label_from_op_comment(submission)
        if labeled_data:
            result_data.append(labeled_data)


    # Print all results at the end (you can also save to a file)
    for entry in result_data:
        print(entry)

    save_to_csv(result_data)

# Run the async main function
await main()

In [ ]:
import os
import csv
import requests
from tqdm import tqdm

# Create folders
base_dir = 'dataset'
os.makedirs(base_dir, exist_ok=True)
for label in ['siblings', 'dating']:
    os.makedirs(os.path.join(base_dir, label), exist_ok=True)

# Read CSV and download images
with open('reddit_posts.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(tqdm(reader)):
        url = row['image_url']
        label = row['label']

        try:
            response = requests.get(url, timeout=10)
            if response.status_code == 200:
                ext = url.split('.')[-1].split('?')[0]
                filename = f"{i:04d}.{ext}"
                path = os.path.join(base_dir, label, filename)

                with open(path, 'wb') as f:
                    f.write(response.content)
            else:
                print(f"Failed to download: {url}")
        except Exception as e:
            print(f"Error downloading {url}: {e}")

112it [00:10, 10.68it/s]


##Instagram

In [ ]:
import os
import re
import csv
import string
from google.colab import drive
from paddleocr import PaddleOCR
from PIL import Image
from IPython.display import display  # Optional for Jupyter/Colab

# Mount Google Drive
drive.mount('/content/drive')

# Initialize PaddleOCR (English, GPU if available)
ocr = PaddleOCR(use_angle_cls=True, lang='en', use_gpu=True)

# Folder with images
IMAGE_FOLDER = "/content/drive/MyDrive/segmentation_dataset/sod"

def clean_and_match_label(text):
    #skip polls
    if "%" in text:
        return None

    # Lowercase and remove punctuation
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    print(f"text: {text}")

    # Normalize repeated letters (e.g., datinggg → dating)
    text = re.sub(r'(.)\1{2,}', r'\1', text)

    # Fuzzy keyword matching for siblings or dating
    if re.search(r'\bsibl\w{0,5}\b', text) or re.search(r'\bsobl\w{0,5}\b', text):
        return "siblings"
    elif re.search(r'\bdati\w{0,5}\b', text) or re.search(r'\bdoot\w{0,5}\b', text):
        return "dating"
    else:
        return None

def extract_label_from_image(image_path):
    try:
        # Extract text with PaddleOCR
        result = ocr.ocr(image_path, cls=True)
        text = " ".join([line[1][0] for line in result[0]]) if result[0] else ""
        return clean_and_match_label(text), text.strip()
    except Exception as e:
        print(f"Error reading {image_path}: {e}")
        return None, None

def group_images_by_post(folder):
    grouped = {}
    for filename in os.listdir(folder):
        if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
            match = re.match(r"(.+?)_(\d+)\.(jpg|jpeg|png)", filename, re.IGNORECASE)
            if match:
                post_id = match.group(1)
                if post_id not in grouped:
                    grouped[post_id] = []
                grouped[post_id].append(filename)
    return grouped

def main():
    output = []
    manual_review = []

    grouped_posts = group_images_by_post(IMAGE_FOLDER)

    for post_id, files in grouped_posts.items():
        raw_image = None
        label = None
        ocr_text_debug = ""

        for file in files:
            if "_1" in file:
                raw_image = file
            elif ("_2" in file or "_3" in file) and not label:
                label_candidate, ocr_text = extract_label_from_image(os.path.join(IMAGE_FOLDER, file))
                ocr_text_debug = ocr_text or ""
                if label_candidate:
                    label = label_candidate

        if raw_image and label:
            print(f"[✓] {raw_image}: {label}")
            output.append((raw_image, label))
        elif raw_image:
            print(f"[!] {raw_image}: could not determine label (OCR saw: '{ocr_text_debug[:60]}')")
            manual_review.append((raw_image, ocr_text_debug))

    # Write successful labels
    with open("/content/drive/MyDrive/classified_posts.csv", "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["filename", "label"])
        for row in output:
            writer.writerow(row)

    # Write uncertain cases for review
    with open("/content/drive/MyDrive/manual_review.csv", "w", newline='') as reviewfile:
        writer = csv.writer(reviewfile)
        writer.writerow(["filename", "ocr_text"])
        for row in manual_review:
            writer.writerow(row)

    print(f"\n✅ Done! Saved {len(output)} labeled entries to 'classified_posts.csv'")
    print(f"🧐 {len(manual_review)} posts need manual review (see 'manual_review.csv')")

if __name__ == "__main__":
    main()

In [ ]:
import csv
import os
import shutil

# Paths
OCR_CSV = "/content/drive/MyDrive/classified_posts.csv"
SOURCE_FOLDER = "/content/drive/MyDrive/segmentation_dataset/sod"
DEST_FOLDER = "/content/dataset"

# Make sure destination subfolders exist
for label in ["siblings", "dating"]:
    os.makedirs(os.path.join(DEST_FOLDER, label), exist_ok=True)

# Copy files based on OCR labels
with open(OCR_CSV, newline='') as f:
    reader = csv.DictReader(f)
    for row in reader:
        filename = row['filename']
        label = row['label'].strip().lower()

        src = os.path.join(SOURCE_FOLDER, filename)
        dest = os.path.join(DEST_FOLDER, label, filename)

        if os.path.exists(src):
            shutil.copyfile(src, dest)
            print(f"[✓] Copied: {filename} → {label}")
        else:
            print(f"[!] File not found: {src}")


[✓] Copied: 2534984807113633820_39225567241_1.jpg → dating
[✓] Copied: 2532080072706728977_39225567241_1.jpg → dating
[✓] Copied: 2536070497838673335_39225567241_1.jpg → siblings
[✓] Copied: 2534737835999912794_39225567241_1.jpg → siblings
[✓] Copied: 2534242119380800755_39225567241_1.jpg → dating
[✓] Copied: 2534367099112375288_39225567241_1.jpg → siblings
[✓] Copied: 2534017243726468582_39225567241_1.jpg → dating
[✓] Copied: 2531137706910962627_39225567241_1.jpg → dating
[✓] Copied: 2530710786406688781_39225567241_1.jpg → siblings
[✓] Copied: 2529086074358425888_39225567241_1.jpg → dating
[✓] Copied: 2528248274549509485_39225567241_1.jpg → dating
[✓] Copied: 2528247091126631780_39225567241_1.jpg → dating
[✓] Copied: 2524681310778438309_39225567241_1.jpg → siblings
[✓] Copied: 2525165956440029018_39225567241_1.jpg → siblings
[✓] Copied: 2526116362641193259_39225567241_1.jpg → siblings
[✓] Copied: 2521315956656204019_39225567241_1.jpg → dating
[✓] Copied: 2521294839677807228_3922556724

In [ ]:
!cp -r /content/dataset /content/drive/MyDrive/sod_dataset

image_paths = [
    "/content/drive/MyDrive/segmentation_dataset/post_text.jpg",  # SIBLINGS!
    "/content/drive/MyDrive/segmentation_dataset/post_poll.jpg"      # Poll
]